In [1]:
import json, os
perm_id = '0'
task_num = 5
class_num = 10
shot_num = 5

def collect_from_json(dataset, root, split):
    default = ['train', 'dev', 'test']
    if split == "train":
        pth = os.path.join(root, dataset, "perm"+perm_id, f"{dataset}_{task_num}task_{class_num // task_num}way_{shot_num}shot.{split}.jsonl")
    elif split in ['dev', 'test']:
        pth = os.path.join(root, dataset, f"{dataset}.{split}.jsonl")
    elif split == "stream":
        pth = os.path.join(root, dataset, f"stream_label_{task_num}task_{class_num // task_num}way.json")
    else:
        raise ValueError(f"Split \"{split}\" value wrong!")
    if not os.path.exists(pth):
        raise FileNotFoundError(f"Path {pth} do not exist!")
    else:
        with open(pth) as f:
            if pth.endswith('.jsonl'):
                data = [json.loads(line) for line in f]
                if split == "train":
                    data = [list(i.values()) for i in data]
            else:
                data = json.load(f)
    return data

In [3]:
from torch.utils.data import Dataset
class MAVEN_Dataset(Dataset):
    def __init__(self, tokens, labels, masks, spans) -> None:
        super(Dataset).__init__()
        self.tokens = tokens
        self.masks = masks
        self.labels = labels
        self.spans = spans
        # self.requires_cl = [0 for _ in range(len(spans))]
        # self.labels = []
        # for stream in streams:
        #     for lb in stream:
        #         if not lb in self.label2idx:
        #             self.label2idx[lb] = len(self.label2idx)
        # for label_ls in labels:
        #     self.labels.append([self.label2idx[label]  for label in label_ls])
    def __getitem__(self, index):
        return [self.tokens[index], self.labels[index], self.masks[index], self.spans[index]]
    def __len__(self):
        return len(self.labels)
    def extend(self, tokens, labels, masks, spans):
        self.tokens.extend(tokens)
        self.labels.extend(labels)
        self.masks.extend(masks)
        self.spans.extend(spans)
        # self.requires_cl.extend(requires_cl)
    # def collate_fn(self, batch):
    #     batch = pad_sequence([torch.LongTensor(item) for item in batch[2]])
    #     pass

In [2]:
import tqdm
import torch
import numpy as np
def collect_dataset(dataset, root, split, label2idx, stage_id, labels):
    if split == 'train':
        data = [instance for t in collect_from_json(dataset, root, split)[stage_id] for instance in t]
    else:
        data = collect_from_json(dataset, root, split)
    data_tokens, data_labels, data_masks, data_spans = [], [], [], []
    for dt in tqdm(data):
        # pop useless properties
        if 'mention_id' in dt.keys():
            dt.pop('mention_id')
        if 'sentence_id' in dt.keys():    
            dt.pop('sentence_id')
        # if split == 'train':
        add_label = []
        add_span = []
        new_t = {}
        for i in range(len(dt['label'])):
            if dt['label'][i] in labels or dt['label'][i] == 0: # if the label of instance is in the query
                add_label.append(dt['label'][i]) # append the instance and the label
                add_span.append(dt['span'][i]) # the same as label
        if len(add_label) != 0:
            token = dt['piece_ids']
            new_t['label'] = add_label
            valid_span = add_span
            valid_label = [label2idx[item] if item in label2idx else 0 for item in add_label]
        # else:
        #     token = dt['piece_ids']
        #     valid_span = dt['span'].copy()
        #     valid_label = [label2idx[item] if item in label2idx else 0 for item in dt['label']]
            # max_seqlen = 90
        max_seqlen = 120 # 344, 249, 230, 186, 167
        if len(token) >= max_seqlen + 2:
            token_sep = token[-1]
            token = token[:max_seqlen + 1] + [token_sep]
            invalid_span = np.unique(np.nonzero(np.asarray(valid_span) > max_seqlen)[0])
            invalid_span = invalid_span[::-1]
            for invalid_idx in invalid_span:
                valid_span.pop(invalid_idx)
                valid_label.pop(invalid_idx)
        if len(token) < max_seqlen + 2:
            token = token + [0] * (max_seqlen + 2 - len(token))
        token_mask = [1 if tkn != 0 else 0 for tkn in token]
            # span_mask = []
            # for i in range(len(token)):
            #     span_mask.append([0, 0])
            # for item in valid_span:
            #     for i in range(len(item)):
            #         span_mask[item[i]][i] = 1
        data_tokens.append(token)
        data_labels.append(valid_label)
        data_masks.append(token_mask)
        data_spans.append(valid_span)
            # data_spans.append(valid_span)
    # if args.my_test:
        return MAVEN_Dataset(data_tokens[:100], data_labels[:100], data_masks[:100], data_spans[:100]) # TODO: deprecated, used for debugging, not for test!

In [4]:
root = './data_incremental'
dataset = 'MAVEN'
split = 'train'


In [5]:
data = collect_from_json(dataset=dataset,root=root,split=split)

In [6]:
len(data)

5

In [7]:
data['label']

TypeError: list indices must be integers or slices, not str

In [16]:
data[0][0][0]['label']

[21, 21, 72, 14, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]